<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
# Import libraries

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [2]:
# Read in data into dataframe

df_trip = pd.read_csv(r'C:\Users\julia\Dropbox\Atlytics Hackathon\trippub.csv')
df_hh = pd.read_csv(r'C:\Users\julia\Dropbox\Atlytics Hackathon\hhpub.csv')
df_veh = pd.read_csv(r'C:\Users\julia\Dropbox\Atlytics Hackathon\vehpub.csv')
df_per = pd.read_csv(r'C:\Users\julia\Dropbox\Atlytics Hackathon\perpub.csv')

In [3]:
# Include only survey particants that own a vehicle

df_veh = df_veh[df_veh['HHVEHCNT']>0]

In [4]:
# Filter desired survey attributes

demograph1 = df_trip[['HOUSEID','VEHID','PERSONID', 'R_SEX_IMP', 'R_AGE_IMP', 'HHFAMINC','LIF_CYC',
                      'EDUC', 'HH_RACE','HOMEOWN','HHSTFIPS', 'HHSIZE', 'HHVEHCNT', 'DRVRCNT', 
                      'WRKCOUNT', 'URBAN', 'PRMACT', 'WORKER']]
demograph2 = df_veh[['HOUSEID','VEHID','HFUEL','VEHAGE','ANNMILES']]
demograph3 = pd.merge(demograph1, demograph2,  how='left', left_on=['HOUSEID','VEHID'], 
                      right_on = ['HOUSEID','VEHID'])

# Remove null values and duplicates from dataframe
demograph3.dropna(inplace=True)
demograph3.drop_duplicates(['HOUSEID','VEHID','PERSONID'], inplace=True)
demograph3.head(10)

,HOUSEID,VEHID,PERSONID,R_SEX_IMP,R_AGE_IMP,HHFAMINC,LIF_CYC,EDUC,HH_RACE,HOMEOWN,...,HHSIZE,HHVEHCNT,DRVRCNT,WRKCOUNT,URBAN,PRMACT,WORKER,HFUEL,VEHAGE,ANNMILES
0,30000007,3,1,2,67,7,10,3,2,1,...,3,5,3,1,1,6,2,-1.0,19.0,900.0
2,30000007,5,2,1,66,7,10,3,2,1,...,3,5,3,1,1,1,1,-1.0,24.0,10000.0
4,30000007,1,3,2,28,7,10,2,2,1,...,3,5,3,1,1,5,2,-1.0,10.0,1000.0
6,30000008,4,1,1,55,8,2,5,1,1,...,2,4,2,2,4,1,1,-1.0,13.0,10000.0
8,30000012,2,1,2,45,10,1,5,1,1,...,1,2,1,1,1,1,1,-1.0,13.0,6000.0
12,30000012,1,1,2,45,10,1,5,1,1,...,1,2,1,1,1,1,1,-1.0,6.0,11000.0
17,30000019,1,1,1,68,3,2,1,1,1,...,2,2,2,0,1,97,2,-1.0,11.0,10000.0
19,30000019,1,2,2,66,3,2,2,1,1,...,2,2,2,0,1,4,2,-1.0,11.0,10000.0
21,30000029,1,1,1,72,5,10,3,1,1,...,2,2,2,0,1,6,2,-1.0,1.0,21058.0
24,30000029,1,2,2,72,5,10,2,1,1,...,2,2,2,0,1,6,2,-1.0,1.0,21058.0


In [7]:
# Rename columns for easier understanding

demograph3.columns = ['HOUSEID', 'VEHID', 'PERSONID', 'Sex', 'Age', 'Family Income',
                      'Life Cycle', 'Education', 'Race', 'Home Ownership', 'City', 'HH Size',
                      'HH Vehicles', 'HH Drivers', 'HH Workers', 'Urban', 'Primary Activity', 
                      'Worker Status', 'BEV', 'Vehicle Age', 'Annual Miles']

In [8]:
demograph3["Education"].replace({-1:1,-7:1,-8:1},inplace=True)##1 is less than high school
#demograph3["Race"].replace({-7:1,-8:1},inplace=True) ##1 is white
demograph3["Home Ownership"].replace({-7:97,-8:97},inplace=True) ##97 is some other arrangement
demograph3["Primary Activity"].replace({-1:97,-7:97,-8:97},inplace=True) ##97 is something else
demograph3["Worker Status"].replace({-1:97,-9:97},inplace=True)##97 is no job
#demograph3["BEV"].replace({-1:97,-8:97,-8:97},inplace=True)##97 is some other fuel
#demograph3["Vehicle Age"].replace({-8:10,-7:10},inplace=True)##10 years old
#demograph3["Annual Miles"].replace({-9:0},inplace=True)##0 miles

In [20]:
demograph3=demograph3[demograph3['Age']>16]

In [24]:
demograph3['Family Income'][demograph3['BEV']==3].value_counts()

 11    160
 10     83
 8      67
 7      53
 9      49
 6      31
 3      12
 4       9
 5       8
-7       5
 1       4
 2       1
Name: Family Income, dtype: int64

In [21]:

demograph3.to_pickle("./demograph3.pkl")

In [10]:
df=pd.read_pickle("./demograph3.pkl")

In [11]:
df['Family Income Code'] = df['Family Income'].copy()
df['Family Income'] = df['Family Income Code'].replace({1: 5000, 2: 12500, 3: 20000, 4: 30000, 
                                                        5: 42500, 6: 62500, 7: 87500, 8: 112500, 
                                                        9: 137500, 10: 175000, 11: 20000})
df['Family Income'].replace({-9: np.nan,-8:np.nan,-7:np.nan} , inplace=True)
df.drop('Family Income Code', axis=1, inplace=True)
df['Family Income'].isnull().sum()

5683

In [12]:
df['Family Income'].describe()

count    202127.000000
mean      73451.975738
std       46959.551965
min        5000.000000
25%       30000.000000
50%       62500.000000
75%      112500.000000
max      175000.000000
Name: Family Income, dtype: float64

In [13]:
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
df['Family Income']=imp_mean.fit_transform(np.array(df['Family Income']).reshape(-1,1))
df['Family Income'].describe()

count    207810.000000
mean      73451.975738
std       46312.994157
min        5000.000000
25%       30000.000000
50%       62500.000000
75%      112500.000000
max      175000.000000
Name: Family Income, dtype: float64

In [16]:
df['Annual Miles'].replace({-9: np.nan,-8:np.nan,-7:np.nan} , inplace=True)
df['Annual Miles'].describe()

count    152839.000000
mean      12038.650482
std       12662.057202
min           0.000000
25%        6000.000000
50%       10000.000000
75%       15000.000000
max      200000.000000
Name: Annual Miles, dtype: float64

In [17]:
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
df['Annual Miles']=imp_mean.fit_transform(np.array(df['Annual Miles']).reshape(-1,1))
df['Annual Miles'].describe()

count    207810.000000
mean      12038.650482
std       10858.948362
min           0.000000
25%        7800.000000
50%       12038.650482
75%       12038.650482
max      200000.000000
Name: Annual Miles, dtype: float64

In [13]:
# Convert BEV to binary: bought BEV (1) or not (0)
df['BEV'] = np.where(df['BEV'] == 3, 1, 0)

df.to_pickle("./demograph.pkl")
#df=pd.read_pickle("./demograph.pkl")

demograph = df.copy().drop(['HOUSEID', 'VEHID', 'PERSONID'],axis=1)


In [14]:
# One hot encode categorical data and standardize numerical features

ss = StandardScaler()
encoder = LabelEncoder()

features = df.columns
features = features[3:]

fit_cols=[]

numerical = ['Age', 'HH Size', 'HH Vehicles', 'HH Drivers', 'HH Workers', 'Family Income', 
            'Vehicle Age', 'Annual Miles']
categorical = list(set(features).difference(set(numerical)))

for feat in categorical:
    df[feat + '_enc'] = encoder.fit_transform(demograph[feat])
    fit_cols.append(feat + '_enc')
    demograph[feat] = encoder.fit_transform(demograph[feat])
    
for feat in numerical:
    df[feat + '_std'] = ss.fit_transform(np.array(demograph[feat]).reshape(-1,1))
    fit_cols.append(feat + '_std')
    demograph[feat] =  ss.fit_transform(np.array(demograph[feat]).reshape(-1,1))

    
demograph.head()


,Sex,Age,Family Income,Life Cycle,Education,Race,Home Ownership,City,HH Size,HH Vehicles,HH Drivers,HH Workers,Urban,Primary Activity,Worker Status,BEV,Vehicle Age,Annual Miles
0,1,0.887915,0.303329,9,2,1,0,33,0.208147,2.234702,1.249077,-0.326856,0,5,1,0,1.809396,-0.658035
2,0,0.840311,0.303329,9,2,1,0,33,0.208147,2.234702,1.249077,-0.326856,0,0,0,0,2.656992,0.094797
4,1,-0.968652,0.303329,9,1,1,0,33,0.208147,2.234702,1.249077,-0.326856,0,4,1,0,0.283723,-0.649762
6,0,0.316664,0.843135,1,4,0,0,49,-0.527457,1.380636,-0.043644,0.705862,3,0,0,0,0.792281,0.094797
8,1,-0.159379,2.192652,0,4,0,0,32,-1.263062,-0.327497,-1.336366,-0.326856,0,0,0,0,0.792281,-0.236118


In [ ]:
demograph3["Education"].replace({-1:1,-7:1,-8:1},inplace=True)##1 is less than high school
demograph3["Race"].replace({-7:1,-8:1},inplace=True) ##1 is white
demograph3["Home Ownership"].replace({-7:97,-8:97},inplace=True) ##97 is some other arrangement
demograph3["Primary Activity"].replace({-1:97,-7:97,-8:97},inplace=True) ##97 is something else
demograph3["Worker Status"].replace({-1:97,-9:97},inplace=True)##97 is no job
demograph3["BEV"].replace({-1:97,-8:97,-8:97},inplace=True)##97 is some other fuel
demograph3["Vehicle Age"].replace({-8:10,-7:10},inplace=True)##10 years old
demograph3["Annual Miles"].replace({-9:0},inplace=True)##0 miles

In [57]:
demograph.to_pickle("./clean_demograph.pkl")